In [ ]:
# Setup OpenAI Agent
import openai

openai.api_key = "sk-your-api-key"
from llama_index.agent import OpenAIAgent

In [ ]:
# Load the OpenAPI spec for OpenAI
import requests
import yaml

f = requests.get(
    "https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/openai.com/1.2.0/openapi.yaml"
).text
open_api_spec = yaml.safe_load(f)

In [ ]:
from llama_index.tools.openapi.base import OpenAPIToolSpec
from llama_index.tools.requests.base import RequestsToolSpec
from llama_index.tools.tool_spec.load_and_search.base import LoadAndSearchToolSpec

open_spec = OpenAPIToolSpec(open_api_spec)
# OR
open_spec = OpenAPIToolSpec(
    url="https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/openai.com/1.2.0/openapi.yaml"
)

requests_spec = RequestsToolSpec(
    {
        "api.openai.com": {
            "Authorization": "Bearer sk-your-key",
            "Content-Type": "application/json",
        }
    }
)

# OpenAPI spec is too large for content, wrap the tool to separate loading and searching
wrapped_tools = LoadAndSearchToolSpec.from_defaults(
    open_spec.to_tool_list()[0],
).to_tool_list()

In [ ]:
agent = OpenAIAgent.from_tools(
    [*wrapped_tools, *requests_spec.to_tool_list()], verbose=True
)

In [ ]:
agent.chat("what is the base url for the server")

=== Calling Function ===
Calling function: load_openapi_spec with args: {}
Got output: Content loaded! You can now search the information using read_load_openapi_spec
=== Calling Function ===
Calling function: read_load_openapi_spec with args: {
  "query": "base url"
}
Got output: 
https://api.openai.com/v1


Response(response='The base URL for the server is `https://api.openai.com/v1`.', source_nodes=[], metadata=None)

In [ ]:
agent.chat("what is the completions api")

=== Calling Function ===
Calling function: read_load_openapi_spec with args: {
  "query": "completions api"
}
Got output: 
The OpenAI API provides a completions API which can be accessed using a POST request to the endpoint '/completions'. This API allows users to generate completions for a given prompt.


Response(response="The completions API is an API provided by the OpenAI API. It can be accessed using a POST request to the endpoint '/completions'. This API allows users to generate completions for a given prompt.", source_nodes=[], metadata=None)

In [ ]:
agent.chat("ask the completions api for a joke")

=== Calling Function ===
Calling function: post_request with args: {
  "url": "https://api.openai.com/v1/completions"
}
Got output: {'error': {'message': 'you must provide a model parameter', 'type': 'invalid_request_error', 'param': None, 'code': None}}


Response(response='To use the completions API and generate a joke, you need to provide a model parameter. Please specify the model you want to use for generating the joke.', source_nodes=[], metadata=None)

In [ ]:
agent.chat("Can you decide the model")

=== Calling Function ===
Calling function: post_request with args: {
  "url": "https://api.openai.com/v1/completions",
  "data": {
    "model": "text-davinci-003",
    "prompt": "Why don't scientists trust atoms?",
    "max_tokens": 50
  }
}
Got output: {'id': 'cmpl-7Yhei5268PoefRhhT47dQgzgmKRg5', 'object': 'text_completion', 'created': 1688505320, 'model': 'text-davinci-003', 'choices': [{'text': '\n\nScientists don’t trust atoms because they are considered to be among the most unpredictable of all known sub-atomic particles and can exist in multiple configurations and arrangements, making it difficult for scientists to predict how they will behave in any given situation', 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 7, 'completion_tokens': 50, 'total_tokens': 57}}


Response(response="Here's a joke for you:\n\nWhy don't scientists trust atoms?\nScientists don’t trust atoms because they are considered to be among the most unpredictable of all known sub-atomic particles and can exist in multiple configurations and arrangements, making it difficult for scientists to predict how they will behave in any given situation.\n\nPlease note that this joke was generated using the text-davinci-003 model.", source_nodes=[], metadata=None)